# **Start of the project**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import all the necessary dependancies****

In [1]:
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score as acc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential,layers
%matplotlib inline

# **Import the training data**

In [2]:
data = pd.read_csv("../input/titanic/train.csv")
data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# **Data Insights**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


# **Some ages are missing and a lot of cabin are missing. We will first predict the ages using linear regression**


In [4]:
def cabin_map(cb):
    if len(cb)>0:
        return cb[0]

# **Mapping non-numeric values to numeric**

In [5]:
def prepro(dt):
    dt['Sex'].replace({'male':0,'female':1},inplace=True)
    dt['Embarked'].replace({'S':0,'C':1,'Q':2},inplace=True)
    dt['Cabin']=dt['Cabin'].astype(str)
    dt['Cabin']=dt['Cabin'].apply(cabin_map)
    dt['Cabin'].replace({'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'T':7,'n':8},inplace=True)
    return dt

In [ ]:
data=prepro(data)
data.head()

# **Preparing data for age predictor linear regression**

In [6]:
dt=data.drop(columns=['PassengerId','Name','Ticket','Fare','Cabin'])
print(len(dt))
dt.dropna(inplace=True)
print(len(dt))
#dt.head()
dt.info()
#dt['Cabin'].value_counts()

891
712
<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  712 non-null    int64  
 1   Pclass    712 non-null    int64  
 2   Sex       712 non-null    object 
 3   Age       712 non-null    float64
 4   SibSp     712 non-null    int64  
 5   Parch     712 non-null    int64  
 6   Embarked  712 non-null    object 
dtypes: float64(1), int64(4), object(2)
memory usage: 44.5+ KB


In [ ]:
dt.head()

In [ ]:
x=dt[['Pclass','Sex','SibSp','Parch','Embarked']]
x=x.iloc[:,:].to_numpy()
y=dt.iloc[:,3].to_numpy()
xtrain,xtest,ytrain,ytest=train_test_split(x,y)
print(xtrain.shape,ytrain.shape)


# **Linear regressor model for age prediction**

In [ ]:
lg_age=LinearRegression(normalize=True)
lg_age.fit(x,y)

# **Filling the null values by predicted values**

In [ ]:
l=len(data)
data['Age'].fillna(-1,inplace=True)
for i in range(l):
    if data.iloc[i]['Age']==-1:
        tmp=data.iloc[i]
        #tmp=tmp[1:3]+tmp[4:5]+tmp[6:8]+tmp[9:10]+tmp[11:]
        tmp=tmp[['Pclass','Sex','SibSp','Parch','Embarked']].to_numpy().reshape(1,-1)
        age_cap=lg_age.predict(tmp)
        data.iloc[i]['Age']=age_cap
data.info()

In [ ]:
data['Embarked'].fillna(3,inplace=True)
data['Embarked']=data['Embarked'].astype('int8')
data['Embarked'].value_counts()

In [ ]:
data.info()

# **Making the first voting cassifier using SVM, logistic regression and KNN**

In [ ]:
dt=data.drop(columns=['PassengerId','Name','Ticket','Fare'])
print(len(dt))
dt.dropna(inplace=True)
print(len(dt))
#dt.head()
dt.info()
#dt['Cabin'].value_counts()

In [ ]:
x=dt[['Pclass','Sex','Age','SibSp','Parch','Cabin','Embarked']]
x=x.iloc[:,:].to_numpy()
y=dt.iloc[:,0].to_numpy()
xtrain,xtest,ytrain,ytest=train_test_split(x,y)
print(xtrain.shape,ytrain.shape)

In [ ]:
log=LogisticRegression()
knn=KNeighborsClassifier(n_neighbors=11)
svm=SVC(gamma='auto',probability=True)
voting_3=VotingClassifier(estimators=[('log',log),('knn',knn),('svc',svm)],voting='soft')

In [ ]:
log.fit(x,y)
knn.fit(x,y)
svm.fit(x,y)


voting_3.fit(x,y)
ypred=voting_3.predict(xtest)
accuracy=acc(ytest,ypred)
print("accuracy=",accuracy)

# Making a random Forest classifier and determine the optimal value of number of estimators


In [ ]:
opt_n=0
opt_acc=0
for i in range(10,500,10):
    rf=RandomForestClassifier(n_estimators=i,random_state=1)
    rf.fit(xtrain,ytrain)
    ypred=rf.predict(xtest)
    accuracy=acc(ytest,ypred)
    #print(i,accuracy)
    if accuracy>opt_acc:
        opt_acc=accuracy
        opt_n=i
print(opt_acc,opt_n)

In [ ]:
rf=RandomForestClassifier(n_estimators=40,random_state=1)
rf.fit(x,y)
y=rf.predict(x)
accuracy=acc(ytest,ypred)
print(accuracy)

# Making a Naive Bayes classifier

In [ ]:
nb=GaussianNB()
nb.fit(x,y)
ypred=rf.predict(xtest)
accuracy=acc(ytest,ypred)
print(accuracy)

In [ ]:
voting_c=VotingClassifier(estimators=[('NB',nb),('log',log),('knn',knn),('svc',svm),('RF',rf)],voting='soft')
voting_c.fit(x,y)
ypred=voting_c.predict(xtest)
accuracy=acc(ytest,ypred)
print("accuracy=",accuracy)

# Making a deep learning model

In [ ]:
def dl():
    model=Sequential()
    model.add(layers.BatchNormalization(input_shape=(7,)))
    model.add(layers.Dense(16,input_shape=(7,),activation='tanh'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(32,input_shape=(7,),activation='tanh'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(64,activation='tanh'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(32,input_shape=(7,),activation='tanh'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(16,activation='tanh'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',  
              metrics=['accuracy'])
    
    return model
    

In [ ]:
model=dl()
model.summary()

In [ ]:
callback=tf.keras.callbacks.EarlyStopping(monitor='loss',min_delta=0.01,patience=50,restore_best_weights=True)
history=model.fit(xtrain,ytrain,batch_size=32,callbacks=[callback],epochs=1000,validation_data=(xtest, ytest))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['train','test'],loc='lower right')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train','test'],loc='upper right')
plt.show()


In [ ]:
ypred=rf.predict(xtest)
accuracy=acc(ytest,ypred)
print('RF:',accuracy)
ypred=log.predict(xtest)
accuracy=acc(ytest,ypred)
print('log:',accuracy)
ypred=knn.predict(xtest)
accuracy=acc(ytest,ypred)
print('KNN:',accuracy)
ypred=svm.predict(xtest)
accuracy=acc(ytest,ypred)
print('SVM:',accuracy)
ypred=nb.predict(x)
accuracy=acc(y,ypred)
print('NB:',accuracy)
# ypred=model.predict(xtest)
# ypred=[1 if i>=0.5 else 0 for i in ypred]
# accuracy=acc(ytest,ypred)
# print('DL:',accuracy)

In [ ]:
ypred=model.predict(xtest)
ypred=[1 if i>=0.5 else 0 for i in ypred]
accuracy=acc(ytest,ypred)
print('DL:',accuracy)

In [ ]:

xgb=GradientBoostingClassifier(n_estimators=100,max_depth=5,random_state=7)
xgb.fit(x,y)
ypred=xgb.predict(xtest)
accuracy=acc(ypred,ytest)
print('Accuracy=',accuracy)

In [ ]:
voting_4=VotingClassifier(estimators=[('XGB',xgb),('NB',nb),('log',log),('knn',knn),('svc',svm),('RF',rf)],voting='soft')
voting_4.fit(x,y)
ypred=voting_4.predict(xtest)
accuracy=acc(ytest,ypred)
print("accuracy=",accuracy)

In [ ]:
voting_2=VotingClassifier(estimators=[('XGB',xgb),('svc',svm),('RF',rf)],voting='soft')
voting_2.fit(x,y)
ypred=voting_2.predict(xtest)
accuracy=acc(ytest,ypred)
print("accuracy=",accuracy)

# Predicting the test set

In [ ]:
test=pd.read_csv('../input/titanic/test.csv')
test=prepro(test)
test.head()

In [ ]:
pid=test['PassengerId'].to_numpy()
print(pid)

In [ ]:
test.drop(columns=['PassengerId','Name','Ticket','Fare'],inplace=True)
test.info()

In [ ]:
l=len(test)
test['Age'].fillna(-1,inplace=True)
for i in range(l):
    if test.iloc[i]['Age']==-1:
        tmp=test.iloc[i]
        #tmp=tmp[1:3]+tmp[4:5]+tmp[6:8]+tmp[9:10]+tmp[11:]
        tmp=tmp[['Pclass','Sex','SibSp','Parch','Embarked']].to_numpy().reshape(1,-1)
        age_cap=lg_age.predict(tmp)
        test.iloc[i]['Age']=age_cap
test.info()

In [ ]:
x_t=test.iloc[:,:].to_numpy()
y_t=xgb.predict(x_t)
result=pd.DataFrame({'PassengerId':pid,'Survived':y_t})
result.head()


In [ ]:
result.to_csv('submission.csv',index=False)